In [1]:
import urllib, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from ast import literal_eval

In [2]:
url = "https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json"
response = urllib.request.urlopen(url)
jsondata = json.loads(response.read())
df=pd.json_normalize(jsondata['features'])
print(df)

         type properties.id   properties.name properties.phone  \
0     Feature    5945030094              德興藥局       03-8889408   
1     Feature    5945040036               黃藥局       03-8267211   
2     Feature    5945050069              富一藥局       03-8531432   
3     Feature    5945050176              永和藥局       03-8573247   
4     Feature    5945051208              祥安藥局       03-8549719   
...       ...           ...               ...              ...   
6300  Feature    2101200017      政大(文山健康服務中心)     (02)22343501   
6301  Feature    0291010010            連江縣立醫院      (083)623995   
6302  Feature    5931033943              崇恩藥局     (02)89214022   
6303  Feature    5939171119              順元藥局      (05)7886555   
6304  Feature    0145080011  衛生福利部花蓮醫院豐濱原住民分院      (03)8358141   

      properties.address  properties.mask_adult  properties.mask_child  \
0      花蓮縣玉里鎮國武里中山路２段５８號                   1725                      0   
1       花蓮縣新城鄉北埔村北埔路２０３號                   1080            

In [3]:
Lat=[]
Lon=[]
a2f=df['geometry.coordinates']
for x in a2f.values:
    Lat.append(x[0])
    Lon.append(x[1])
df['Lat']=Lat
df['Lon']=Lon
print(df)

         type properties.id   properties.name properties.phone  \
0     Feature    5945030094              德興藥局       03-8889408   
1     Feature    5945040036               黃藥局       03-8267211   
2     Feature    5945050069              富一藥局       03-8531432   
3     Feature    5945050176              永和藥局       03-8573247   
4     Feature    5945051208              祥安藥局       03-8549719   
...       ...           ...               ...              ...   
6300  Feature    2101200017      政大(文山健康服務中心)     (02)22343501   
6301  Feature    0291010010            連江縣立醫院      (083)623995   
6302  Feature    5931033943              崇恩藥局     (02)89214022   
6303  Feature    5939171119              順元藥局      (05)7886555   
6304  Feature    0145080011  衛生福利部花蓮醫院豐濱原住民分院      (03)8358141   

      properties.address  properties.mask_adult  properties.mask_child  \
0      花蓮縣玉里鎮國武里中山路２段５８號                   1725                      0   
1       花蓮縣新城鄉北埔村北埔路２０３號                   1080            

In [4]:
a1f=df[(df['properties.county']=="臺北市")|(df['properties.county']=="新北市")|(df['properties.county']=="桃園市")]
a1f['properties.county']=a1f['properties.county'].map({'臺北市':1,'新北市':2,'桃園市':3})
print(a1f)

         type properties.id properties.name properties.phone  \
265   Feature    5931111988            滿益藥局      02-86421562   
305   Feature    5901023653          信安健康藥局      02-27084858   
918   Feature    5932076148            宏仁藥局       03-3506590   
919   Feature    5932076157            安康藥局       03-3597032   
920   Feature    5932076255           安得福藥局      02-82001467   
...       ...           ...             ...              ...   
6297  Feature    2101191068        萬華健康服務中心     (02)23395384   
6298  Feature    2101020019        大安健康服務中心     (02)27390997   
6299  Feature    2101090011        大同健康服務中心     (02)25948971   
6300  Feature    2101200017    政大(文山健康服務中心)     (02)22343501   
6302  Feature    5931033943            崇恩藥局     (02)89214022   

      properties.address  properties.mask_adult  properties.mask_child  \
265        新北市汐止區仁愛路１３０號                   3456                     50   
305   臺北市大安區仁愛路４段１２２巷５８號                   4950                    270   
918     桃

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
from sklearn import svm
#from sklearn.calibration import CalibratedClassifierCV
X=a1f['geometry.coordinates'].values.tolist()
y=a1f['properties.county']

In [ ]:
#SVM
def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min() , x.max() 
    y_min, y_max = y.min() , y.max() 
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

C = 1.0  # SVM regularization parameter
models = (svm.SVC(kernel='linear', C=C,probability=True),
          #CalibratedClassifierCV(svm.LinearSVC(C=C, max_iter=10000)),
          #svm.LinearSVC(C=C, max_iter=10000),
          svm.SVC(kernel='rbf', gamma=0.7, C=C,probability=True),
          svm.SVC(kernel='poly', degree=3, gamma='auto', C=C,probability=True))
models = (clf.fit(X, y) for clf in models)

# title for the plots
titles = ('SVC with linear kernel',
          #'LinearSVC (linear kernel)',
          'SVC with RBF kernel',
          'SVC with polynomial (degree 3) kernel')

# Set-up 2x2 grid for plotting.
fig, sub = plt.subplots(2, 2,figsize=(20, 12))
plt.subplots_adjust(wspace=0.5, hspace=0.5)

X0, X1 = a1f['Lat'].to_numpy(), a1f['Lon'].to_numpy()
xx, yy = make_meshgrid(X0, X1)

for clf, title, ax in zip(models, titles, sub.flatten()):
    plot_contours(ax, clf, xx, yy,
                  cmap=plt.cm.coolwarm, alpha=0.8)
    ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel('Lat')
    ax.set_ylabel('Lon')
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(title)
    
    
    #print(clf.predict(np.c_[[25.018796], [121.408557]]))
    #print( clf.predict_proba(np.c_[[25.018796], [121.408557]]))
plt.show()

In [ ]:
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import (KNeighborsClassifier,NeighborhoodComponentsAnalysis)
from sklearn.pipeline import Pipeline
X=[]
y=a1f['properties.county']
X0, X1 = a1f['Lat'].to_numpy(), a1f['Lon'].to_numpy()
for i in range(0,len(X0)):
    X.append([X0[i],X1[i]])
X=np.array(X)
n_neighbors = 1
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, stratify=y, test_size=0.7, random_state=42)

h = .01  # step size in the mesh

# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

names = ['KNN', 'NCA, KNN']

classifiers = [Pipeline([('scaler', StandardScaler()),
                         ('knn', KNeighborsClassifier(n_neighbors=n_neighbors))
                         ]),
               Pipeline([('scaler', StandardScaler()),
                         ('nca', NeighborhoodComponentsAnalysis()),
                         ('knn', KNeighborsClassifier(n_neighbors=n_neighbors))
                         ])
               ]

x_min, x_max = X[:, 0].min(), X[:, 0].max()
y_min, y_max = X[:, 1].min(), X[:, 1].max()
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

for name, clf in zip(names, classifiers):

    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light, alpha=.8)

    # Plot also the training and testing points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold, edgecolor='k', s=20)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("{} (k = {})".format(name, n_neighbors))
    plt.text(0.9, 0.1, '{:.2f}'.format(score), size=15,
             ha='center', va='center', transform=plt.gca().transAxes)
    #print(clf.predict([25.021829, 121.411626]))25.018796, 121.408557
    
    print(clf.predict(np.c_[[25.018796], [121.408557]]))
plt.show()
